In [ ]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Building an ADK agent using QWEN 3 on Vertex AI

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/open-models/agents/qwen3_adk_vertexai.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fopen-models%2Fagents%2Fqwen3_adk_vertexai.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/open-models/agents/qwen3_adk_vertexai.ipynb">
      <img src="https://www.gstatic.com/images/branding/gcpiconscolors/vertexai/v1/32px.svg" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/open-models/agents/qwen3_adk_vertexai.ipynb">
      <img width="32px" src="https://www.svgrepo.com/download/217753/github.svg" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

<div style="clear: both;"></div>

<b>Share to:</b>

<a href="https://www.linkedin.com/sharing/share-offsite/?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/open-models/agents/qwen3_adk_vertexai.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/8/81/LinkedIn_icon.svg" alt="LinkedIn logo">
</a>

<a href="https://bsky.app/intent/compose?text=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/open-models/agents/qwen3_adk_vertexai.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/7/7a/Bluesky_Logo.svg" alt="Bluesky logo">
</a>

<a href="https://twitter.com/intent/tweet?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/open-models/agents/qwen3_adk_vertexai.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/5a/X_icon_2.svg" alt="X logo">
</a>

<a href="https://reddit.com/submit?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/open-models/agents/qwen3_adk_vertexai.ipynb" target="_blank">
  <img width="20px" src="https://redditinc.com/hubfs/Reddit%20Inc/Brand/Reddit_Logo.png" alt="Reddit logo">
</a>

<a href="https://www.facebook.com/sharer/sharer.php?u=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/open-models/agents/qwen3_adk_vertexai.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/51/Facebook_f_logo_%282019%29.svg" alt="Facebook logo">
</a>

| Author(s) |
| --- |
| [Firstname Lastname](https://github.com/username) |

## Overview

This tutorial demonstrates how to build an AI agent using Agent Development Kit (ADK) with Qwen3 model deployed on Vertex AI. Also, it shows how to deploy the agent as a scalable, production-ready service with Vertex AI Agent Engine.

What you'll learn how to


*   Deploy an open-source model like Qwen3 from Vertex AI Model Garden to a dedicated endpoint.

*   Build a tool-calling agent using Google's Agent Development Kit (ADK).

*   Integrate a custom model endpoint with ADK using the LiteLLM library.

*   Define tools and provide instructions for the agent to use them correctly.

*   Test the agent's functionality in a local environment.

*   Package and deploy the ADK agent to Vertex AI Agent Engine for a scalable, production-ready implementation.


## Get started

### Install required packages

First things first, let's get our toolkit ready. We'll need to install the necessary Python packages for interacting with Vertex AI, ADK, and LiteLLM.

In [ ]:
%pip install --upgrade --quiet "google-cloud-aiplatform[adk,agent_engines]>=1.101.0" "litellm>=1.73.6.post1" "nest-asyncio>=1.6.0"

### Authenticate your notebook environment (Colab only)

If you're running this notebook on Google Colab, run the cell below to authenticate your environment.

In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud project information

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [ ]:
# Use the environment variable if the user doesn't provide Project ID.
import os
import uuid

import vertexai

# Project settings
PROJECT_ID = "[your-project-id]"  # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}
if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))
if not PROJECT_ID:
    raise ValueError("PROJECT_ID not set. Please set it in the cell above.")

LOCATION = os.environ.get("GOOGLE_CLOUD_REGION", "us-central1")
BUCKET_URI = "[your-bucket-uri]"  # @param {type: "string", placeholder: "[your-bucket-uri]", isTemplate: true}
if not BUCKET_URI or BUCKET_URI == "[your-bucket-uri]":
    raise ValueError("Please set a valid GCS bucket URI in the BUCKET_URI variable.")

# Set config for adk
os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID
os.environ["GOOGLE_CLOUD_LOCATION"] = LOCATION
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "1"

# Agent configuration
APP_NAME = "weather_agent"
USER_ID = "user_" + str(uuid.uuid4())
SESSION_ID = "session_" + str(uuid.uuid4())
MAX_TOKENS = 1000
TEMPERATURE = 1.0

# Initialize Vertex AI
vertexai.init(project=PROJECT_ID, location=LOCATION, staging_bucket=BUCKET_URI)

### Import libraries

Now, let's import all the Python libraries and modules we'll be using throughout this tutorial. This includes components from Vertex AI, ADK, LiteLLM, and standard Python libraries.


In [ ]:
import functools
import inspect
from typing import Callable, Optional, get_type_hints

import google.auth
import nest_asyncio
from google import adk
from google.adk.agents import Agent
from google.adk.models.lite_llm import LiteLlm
from google.adk.sessions import InMemorySessionService
from google.adk.tools import FunctionTool
from google.cloud import aiplatform
from google.genai import types
from litellm import completion
from vertexai import agent_engines
from vertexai.preview import model_garden, reasoning_engines

nest_asyncio.apply()

### Helpers

We'll define a few helper functions to simplify running our agent and interacting with it in a chat-like interface. These helpers will allow us to test the agent both in the local runner and on the remote Agent Engine.

In [ ]:
def python_type_to_json_schema_type(py_type):
    """Maps Python types to JSON schema types."""
    if py_type in (int, float):
        return "integer" if py_type is int else "number"
    elif py_type is str:
        return "string"
    elif py_type is bool:
        return "boolean"
    elif py_type is list:
        return "array"
    elif py_type is dict:
        return "object"
    return "string"  # Default to string for unknown types


def convert_to_openai_tool(func):
    """
    Converts a Python function into an OpenAI tool schema.
    """
    # Get the function's signature and docstring
    signature = inspect.signature(func)
    docstring = inspect.getdoc(func) or ""

    # Get type hints
    type_hints = get_type_hints(func)

    # Initialize the schema
    function_schema = {
        "name": func.__name__,
        "description": docstring.split("\n\n")[0],
        "parameters": {
            "type": "object",
            "properties": {},
            "required": [],
        },
    }

    # Process each parameter
    for param_name, param in signature.parameters.items():
        param_type = type_hints.get(param_name, str)
        json_type = python_type_to_json_schema_type(param_type)

        function_schema["parameters"]["properties"][param_name] = {
            "type": json_type,
            "description": f"The {param_name} for the function.",
        }

        # Check if the parameter is required
        if param.default is inspect.Parameter.empty:
            function_schema["parameters"]["required"].append(param_name)

    # Final tool schema structure
    tool_schema = {"type": "function", "function": function_schema}

    return tool_schema


def get_base_url(endpoint: aiplatform.Endpoint) -> str:
    """Get the base URL for the endpoint."""
    location = endpoint.resource_name.split("/")[3]
    if endpoint.gca_resource.dedicated_endpoint_enabled:
        return (
            f"https://{endpoint.gca_resource.dedicated_endpoint_dns}"
            f"/v1beta1/{endpoint.resource_name}"
        )
    return (
        f"https://{location}-aiplatform.googleapis.com"
        f"/v1beta1/{endpoint.resource_name}"
    )


def run_single_turn(
    runner: adk.Runner, query: str, session_id: str, user_id: str
) -> Optional[str]:
    """Runs one turn with a local ADK runner."""
    content = types.Content(role="user", parts=[types.Part(text=query)])
    events = runner.run(user_id=user_id, session_id=session_id, new_message=content)
    for event in events:
        if event.is_final_response():
            return event.content.parts[0].text
    return None


def run_remote_single_turn(
    remote_app: reasoning_engines.templates.adk.AdkApp,
    query: str,
    session_id: str,
    user_id: str,
) -> Optional[str]:
    """Runs one turn on a deployed Agent Engine."""
    events = remote_app.stream_query(
        user_id=user_id, session_id=session_id, message=query
    )
    final_event = [event for event in events][-1]
    return final_event["content"]["parts"][0]["text"]


def chat_loop(
    turn_handler: Callable[[str, str, str], Optional[str]],
    session_id: str,
    user_id: str,
) -> None:
    """Main chat interface loop. It is given the function to handle each turn."""
    print("\nStarting chat. Type 'exit' or 'quit' to end.")
    while True:
        try:
            user_input = input("You: ")
            if user_input.lower() in ["quit", "exit", "bye"]:
                print("\nAssistant: Thank you for chatting. Have a great day!")
                break

            response = turn_handler(user_input, session_id, user_id)
            if response:
                print(f"Assistant: {response}")
            else:
                print("Assistant: I received no response.")
        except (KeyboardInterrupt, EOFError):
            print("\nAssistant: Chat ended.")
            break

## Deploy Qwen3 model using Vertex AI Model Garden SDK

Deploy the Qwen3 model from the Vertex AI Model Garden. Model Garden provides a curated collection of foundation models that can be easily deployed to a Vertex AI Endpoint. This gives us a scalable, managed API for our model, which our agent will use as its "brain."


In [ ]:
# Load the model from Model Garden
model = model_garden.OpenModel("publishers/qwen/models/qwen3@qwen3-1.7b")

# Get container specification
serving_container_spec = model.list_deploy_options()[0].container_spec

# Deploy the model
endpoint = model.deploy(
    serving_container_spec=serving_container_spec,
    machine_type="g2-standard-12",
    accelerator_type="NVIDIA_L4",
    accelerator_count=1,
    use_dedicated_endpoint=False,
    spot=False,
    deploy_request_timeout=1800
)

## Test the deployed endpoint with a simple request using LiteLlm

Before building the full agent, it's a good practice to ensure we can communicate with our newly deployed Qwen3 endpoint. We'll use the LiteLLM library for this.

LiteLLM provides a simple, unified interface to call over 100 different LLM APIs. It allows us to interact with our custom Vertex AI endpoint using the same syntax as the OpenAI API. This is incredibly useful because the ADK has a built-in LiteLlm model wrapper, making integration seamless.

Get the necessary authentication credentials for our Google Cloud project.

In [ ]:
# Get credentials
creds, _ = google.auth.default()
auth_req = google.auth.transport.requests.Request()
creds.refresh(auth_req)

Define a simple Python function (get_weather) that we'll later turn into a tool for our agent.


In [ ]:
# Define a simple tool
def get_weather(city: str) -> str:

    """
    Get weather information for a city.

    Args:
        city: The city name to get weather for.

    Returns:
        Weather information string.
    """
    city_lower = city.lower()

    # Simple weather simulation
    weather_data = {
        "san francisco": "It's 70°F (21°C) and foggy.",
        "sf": "It's 70°F (21°C) and foggy.",
        "new york": "It's 75°F (24°C) and partly cloudy.",
        "nyc": "It's 75°F (24°C) and partly cloudy.",
        "london": "It's 60°F (15°C) and rainy.",
        "tokyo": "It's 72°F (22°C) and sunny.",
    }

    for key, weather in weather_data.items():
        if key in city_lower:
            return weather

    return f"It's 80°F (27°C) and sunny in {city}."

Use litellm.completion to send a direct request to our Qwen3 endpoint (vertex_ai/openai/{endpoint.name}). This verifies that the endpoint is live and that our connection is configured correctly.

In [ ]:
response = completion(
    model=f"vertex_ai/openai/{endpoint.name}",
    messages=[{"content": "Hello, how are you?/no_think", "role": "user"}],
    tools=[
        # Convert the function to an OpenAI tool schema
        convert_to_openai_tool(get_weather)
    ],
    temperature=0.7,
    max_tokens=8192,
    extra_body={
        "chat_template_kwargs": {"enable_thinking": True},
    },
)

print(response["choices"][0]["message"]["content"])

## Create an Agent with ADK

Now that we have a working model endpoint, it's time to build our agent using the Agent Development Kit. An ADK Agent is defined by several key components:


*  model: The LLM the agent will use. Here, we'll use ADK's LiteLlm wrapper to point to our Qwen3 endpoint.
*   name: A unique identifier for the agent.

*   description: A high-level summary of what the agent does.
*   instruction: The core prompt that guides the agent's behavior. This is where we tell the agent how and when to use its tools.
*   tools: A list of functions the agent can call to interact with external systems. We'll use the FunctionTool wrapper to make our get_weather function available to the agent.


In [ ]:
root_agent = Agent(
    model=LiteLlm(
        model=f"vertex_ai/openai/{endpoint.name}",
    ),
    name="weather_agent",
    description="Provides weather information for specific cities.",
    instruction="You are a helpful weather assistant. "
    "When the user asks for the weather in a specific city, "
    "use the 'get_weather' tool to find the information. "
    "If the tool returns an error, inform the user politely. "
    "If the tool is successful, present the weather report clearly."
    "After using tools, provide a natural language response incorporating the results."
    "/no_think",
    tools=[FunctionTool(func=get_weather)],
)

## Test the agent locally

Before deploying our agent to a production environment, it's crucial to test its logic and behavior locally. The ADK Runner allows us to do this easily.

The Runner orchestrates the interaction between the user, the agent, and the various services. For this local test, we'll use an InMemorySessionService to keep track of the conversation history.

The chat_loop helper function will provide a simple command-line interface to interact with our agent and see how it responds and uses its get_weather tool.

In [ ]:
# Create a session to store conversations
session_service = InMemorySessionService()

# Create the specific session where the conversation will happen
session = await session_service.create_session(
    app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID
)

# Create the runner to run the agent
runner = adk.Runner(
    agent=root_agent, app_name=APP_NAME, session_service=session_service
)

local_turn_handler = functools.partial(run_single_turn, runner)

chat_loop(local_turn_handler, session.id, USER_ID)

## Deploy the agent on Agent Engine

Once we're satisfied with our agent's local performance, the final step is to deploy it as a scalable, production-ready service using Vertex AI Agent Engine. This powerful platform takes our ADK-defined agent and hosts it as a managed endpoint, handling all the underlying infrastructure, scaling, and logging.

### Prepare your agent for Agent Engine

To prepare our agent for deployment, we wrap it in a reasoning_engines.AdkApp. This class serves as the bridge between the ADK framework and the Agent Engine deployment platform.


In [ ]:
app = reasoning_engines.AdkApp(
    agent=root_agent,
    enable_tracing=True,
)

### Send queries to your agent (locally)

Before pushing to the cloud, we can use the AdkApp object to run a final local test. This ensures that the agent is packaged correctly and behaves as expected within the Agent Engine wrapper.


In [ ]:
session = app.create_session(user_id=USER_ID)

local_app_turn_handler = functools.partial(run_remote_single_turn, app)

chat_loop(local_app_turn_handler, session.id, USER_ID)

### Deploy your agent to Agent Engine

Deploying is as simple as calling the agent_engines.create() function. We pass our AdkApp object and specify any Python package requirements. Agent Engine handles the rest: it containerizes the application, provisions the necessary resources, and exposes a secure endpoint for our agent.

Note: The deployment process can take a few minutes.


In [ ]:
remote_app = agent_engines.create(
    agent_engine=app,
    requirements=[
        "google-cloud-aiplatform[adk,agent_engines]>=1.101.0",
        "litellm==1.73.6.post1",
    ],
)

### Send queries to your agent (remote)

With the agent deployed, we get a remote_app object that points to our live Agent Engine endpoint. We can interact with this remote agent using the exact same chat_loop function, demonstrating the seamless transition from local development to cloud deployment.

In [ ]:
remote_session = remote_app.create_session(user_id=USER_ID)

remote_app_turn_handler = functools.partial(run_remote_single_turn, remote_app)

chat_loop(remote_app_turn_handler, remote_session.id, USER_ID)

## Cleaning up

To avoid incurring ongoing charges to your Google Cloud account, it's important to clean up the resources you've created. The following code will delete the deployed Agent Engine and the Vertex AI Endpoint for the Qwen3 model.


In [ ]:
delete_endpoint = False
delete_agent_engine = False

if delete_endpoint:
    endpoint.delete(force=True)

if delete_agent_engine:
    remote_app.delete(force=True)